In [65]:
import requests, re
from bs4 import BeautifulSoup

In [66]:
url = "https://delhihighcourt.nic.in/app/"

In [78]:
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

captcha_code = soup.find('span', class_ = 'captcha-code').text.strip()
print('This is code:', captcha_code)

options = soup.select('select#year option')
caseYears = [int(opt['value']) for opt in options if opt['value'] != '']
print('These are the years:', caseYears)

caseTypes = [
    option["value"]
    for option in soup.select("select#case_type option")
    if option["value"].strip() != ""]
print('These are the types:', caseTypes)

scripts = soup.find_all("script")
for script in scripts:
    if script.string and "_token" in script.string:
        match = re.search(r'"_token"\s*:\s*"([^"]+)"', script.string)
        if match: token = match.group(1); break
print('This is the token:', token)

This is code: 4209
These are the years: [2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951]
These are the types: ['ADMIN.REPORT', 'ARB.A.', 'ARB. A. (COMM.)', 'ARB.P.', 'BAIL APPLN.', 'CA', 'CA (COMM.IPD-CR)', 'C.A.(COMM.IPD-GI)', 'C.A.(COMM.IPD-PAT)', 'C.A.(COMM.IPD-PV)', 'C.A.(COMM.IPD-TM)', 'CAVEAT(CO.)', 'CC(ARB.)', 'CCP(CO.)', 'CCP(REF)', 'CEAC', 'CEAR', 'CHAT.A.C.', 'CHAT.A.REF', 'CMI', 'CM(M)', 'CM(M)-IPD', 'C.O.', 'CO.APP.', 'CO.APPL.(C)', 'CO.APPL.(M)', 'CO.A(SB)', 'C.O.(COMM.IPD-CR)', 'C.O.(COMM.IPD-GI)', 'C.O.(COMM.IPD-PAT)', 'C.O. (COMM.IPD-TM)', 'CO.EX.', 'CONT.APP.(C)', 'CONT.CAS(

In [82]:
def fetch_case_data(case_type, case_number, filing_year):
    session = requests.Session()

    URL = "https://delhihighcourt.nic.in/app"
    home_res = session.get(URL)
    home_res.raise_for_status()
    soup = BeautifulSoup(home_res.text, "html.parser")

    scripts = soup.find_all("script")
    token = None
    for script in scripts:
        if script.string and "_token" in script.string:
            match = re.search(r'"_token"\s*:\s*"([^"]+)"', script.string)
            if match:
                token = match.group(1)
                break
    if not token:
        raise Exception("CSRF token not found.")

    captcha_span = soup.find('span', class_='captcha-code')
    if not captcha_span:
        raise Exception("Captcha code not found on the page.")
    captchaCode = captcha_span.text.strip()

    captcha_response = session.post(
        "https://delhihighcourt.nic.in/app/validateCaptcha",
        data={
            "_token": token,
            "captchaInput": captchaCode
        }
    )
    captcha_response.raise_for_status()
    captcha_json = captcha_response.json()

    if captcha_json.get("success"):
        data_response = session.post(
            "https://delhihighcourt.nic.in/app/get_casewise_petname",
            data={
                "_token": token,
                "case_type": case_type,
                "case_number": case_number,
                "year": filing_year
            }
        )
        print(data_response.text)
    else:
        print("❌ CAPTCHA failed")

    return 'done'

In [83]:
fetch_case_data(case_type='CM(M)', case_number='676', filing_year=2023)

<div style='text-align: center;'><div style='margin-top: 15px; padding: 10px; border: 1px solid #ccc; border-radius: 5px; background-color: #f9f9f9;'><p><strong style='font-size: .8em;'>Title:</strong> <span class='text-primary' style='font-size: .8em;'>M/S BELA CREATION PVT. LTD VS. MS. SHELLY SYAL</span></p><p'><span style='margin-right: 15px;'><strong style='font-size: .8em;'>Status:</strong> <span class='text-primary' style='font-size: .8em;'>D </span></span><span style='font-size: .8em;'><strong>Next Date:</strong> <span class='text-primary' style='font-size: .8em;'>NA</span></span><p> <p>
            <a href='https://delhihighcourt.nic.in/app/online-cause-history/eyJpdiI6IkVPSDlPZ2d2VkZBZ3dDZ2h1dC8vbVE9PSIsInZhbHVlIjoiZWRPTUJUR25YUWxaNlQ3OUxHU3JTQT09IiwibWFjIjoiZjU2MzRiYWRmNjllMmEzODA0ZWFkNjJkYjc5ZjYxNDQ2Y2MwY2M5OGU5M2Y0ZmJmYzdlYzNjYzNmOTFmMjZjMyIsInRhZyI6IiJ9/eyJpdiI6IlhtWUhJd29Td2dSR1FwKzYzN1NRU2c9PSIsInZhbHVlIjoiV0ZoZk41bk9yK2RUVTdGSkRJYkwydz09IiwibWFjIjoiZWJjMGUzZDI1YTRkODZjM

'done'

In [86]:
import requests

url = "https://delhihighcourt.nic.in/app/"

params = {
    "draw": 1,
    "columns[0][data]": "DT_RowIndex",
    "columns[0][name]": "DT_RowIndex",
    "columns[0][searchable]": "true",
    "columns[0][orderable]": "false",
    "columns[0][search][value]": "",
    "columns[0][search][regex]": "false",
    "columns[1][data]": "case_no_order_link",
    "columns[1][name]": "case_no_order_link",
    "columns[1][searchable]": "true",
    "columns[1][orderable]": "true",
    "columns[1][search][value]": "",
    "columns[1][search][regex]": "false",
    "columns[2][data][_]" : "order_date.display",
    "columns[2][data][sort]" : "order_date.timestamp",
    "columns[2][name]": "order_date.timestamp",
    "columns[2][searchable]": "true",
    "columns[2][orderable]": "true",
    "columns[2][search][value]": "",
    "columns[2][search][regex]": "false",
    "columns[3][data]": "corrigendum",
    "columns[3][name]": "corrigendum",
    "columns[3][searchable]": "true",
    "columns[3][orderable]": "true",
    "columns[3][search][value]": "",
    "columns[3][search][regex]": "false",
    "columns[4][data]": "hindi_order",
    "columns[4][name]": "hindi_order",
    "columns[4][searchable]": "true",
    "columns[4][orderable]": "true",
    "columns[4][search][value]": "",
    "columns[4][search][regex]": "false",
    "order[0][column]": 0,
    "order[0][dir]": "asc",
    "order[0][name]": "DT_RowIndex",
    "start": 0,
    "length": 50,
    "case_type": "CM(M)",       # Fill if needed
    "case_number": "676",     # Fill if needed
    "year": "2023",            # Fill if needed
}

headers = {
    "Referer": "https://delhihighcourt.nic.in/app/sitting-judge-wise",
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0",
}

r = requests.get(url, headers=headers, params=params)
r = r.json()

In [87]:
r['data']

[{'pno': 162553,
  'caseno': 'CM(M) 676/2023',
  'orddate': '26/04/2024',
  'orderdate': '2024-04-26 00:00:00',
  'jud1': '616',
  'jud2': '',
  'jud3': '',
  'pdffilename': '100018521714649800783_55497_6762023.pdf',
  'pdfbilingualfilename': None,
  'ordertext': None,
  'case_no_order_link': "<a href='https://delhihighcourt.nic.in/app/showlogo/100018521714649800783_55497_6762023.pdf/2024' target='_blank'>CM(M) 676/2023</a>",
  'order_date': {'display': '26-04-2024', 'timestamp': 1714069800},
  'corrigendum': 'NA',
  'hindi_order': '',
  'DT_RowIndex': 1},
 {'pno': 796149,
  'caseno': 'CM(M) 676/2023',
  'orddate': '05/10/2023',
  'orderdate': '2023-10-05 00:00:00',
  'jud1': '589',
  'jud2': '',
  'jud3': '',
  'pdffilename': '525552171696666826154_27175_2023.pdf',
  'pdfbilingualfilename': None,
  'ordertext': None,
  'case_no_order_link': "<a href='https://delhihighcourt.nic.in/app/showlogo/525552171696666826154_27175_2023.pdf/2023' target='_blank'>CM(M) 676/2023</a>",
  'order_date